# The ML Pipeline for a Model to Predict Length of Hospital Delivery Stay

In [2]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import matplotlib
from math import ceil
import pickle

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import PredefinedSplit

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import ParameterGrid

from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import xgboost

# Gather Data

In [3]:
# specifying data types for the columns to maintain formatting from original data
data_types = {
    'hospital_service_area': object, 
    'hospital_county': object,
    'operating_certificate_number': object, 
    'permanent_facility_id': object,
    'facility_name': object, 
    'age_group': object, 
    'zip_code_3_digits': object, 
    'gender': object, 
    'race': object,
    'ethnicity': object, 
    'payment_typology_1': object, 
    'payment_typology_2': object,
    'payment_typology_3': object, 
    'length_of_stay': int
}

In [4]:
all_visits = pd.read_csv('../data/planned_deliveries.csv', dtype=data_types)
all_visits = all_visits.loc[:, all_visits.columns != 'Unnamed: 0']
y = all_visits['length_of_stay']
X = all_visits.loc[:, all_visits.columns != 'length_of_stay']

# Compute Baseline Score

In [6]:
mean_length_of_stay = np.mean(y)
median_length_of_stay = np.median(y)
print('Mean length of stay:', mean_length_of_stay)
print('Median length of stay:', median_length_of_stay)

Mean length of stay: 2.3057860029780897
Median length of stay: 2.0


#### R^2 [dimensionless]  
**Pretty sure this is unnecessary to do though since by definition R^2 should = 0 for the expected (average) guess of y**

In [7]:
y_pred_mean = pd.Series([mean_length_of_stay]*len(y))

In [8]:
baseline_r2 = r2_score(y, y_pred_mean)
print('Baseline R^2 [dimensionless]:', baseline_r2)

Baseline R^2 [dimensionless]: 0.0


# Split, Train, and Cross Validate - R^2 Evaluation Metric

### Set up functions for automated pipeline

In [ ]:
def stratified_continuous_split(X:pd.DataFrame, y:pd.Series, train_size:float, val_size:float, test_size:float, random_state:int):
    '''
    Performs a stratified split of inputted data (with respect to y) into a training set, validation set, and test set to specified percentages 
    of the data using verstack's scsplit and performs basic error checking.

    Parameters:
    - X: a 2D pandas DataFrame, the feature matrix
    - y: a 1D pandas Series, the target variable matrix matching X
    - train_size: a float between 0 and 1, the percentage of X which should be training data
    - val_size: a float between 0 and 1, the percentage of X which should be reserved for validation
    - test_size: a float between 0 and 1, the percentage of X which should be reserved for final testing
    - random_state: an int, the random state to split with
    Note: The sum of train_size + val_size + test_size must be 1.0 (100% of X).

    Returns:
    - (X_train) a 2D pandas DataFrame, the feature matrix of training data
    - (y_train) a 1D pandas Series, the target variable matrix for training data
    - (X_val) a 2D pandas DataFrame, the feature matrix of validation data
    - (y_val) a 1D pandas Series, the target variable matrix for validation data
    - (X_test) a 2D pandas DataFrame, the feature matrix of testing data
    - (y_test) a 1D pandas Series, the target variable matrix for testing data

    Raises:
    - ValueError for invalid input
    '''
    from verstack.stratified_continuous_split import scsplit
    
    if ((train_size + val_size + test_size) != 1):
        raise ValueError('Your train_size + val_size + test_size must add up to 1 (100%)!')
    if (not isinstance(random_state, int)):
        raise ValueError('Your random_state must be an int!')

    X_train, X_other, y_train, y_other = scsplit(X, y, stratify=y, test_size=(1-train_size), random_state=random_state)
    
    X_len = X.shape[0]
    test_percent_of_other = (test_size * X_len)/(X_len - (train_size * X_len))
    X_other = X_other.reset_index(drop=True)
    y_other = y_other.reset_index(drop=True)
    
    X_val, X_test, y_val, y_test = scsplit(X_other, y_other, stratify=y_other, test_size=test_percent_of_other, random_state=random_state)

    # basic error checking to check that split returned train, val, and test of expected sizes
    train_count_low = (int)(train_size * X_len)
    train_count_high = ceil(train_size * X_len)
    val_count_low = (int)(val_size * X_len)
    val_count_high = ceil(val_size * X_len)
    test_count_low = (int)(test_size * X_len)
    test_count_high = ceil(test_size * X_len)
    
    Xtrain_fin = X_train.shape[0]
    ytrain_fin = y_train.shape[0]
    Xval_fin = X_val.shape[0]
    yval_fin = y_val.shape[0]
    Xtest_fin = X_test.shape[0]
    ytest_fin = y_test.shape[0]
    
    if not (((Xtrain_fin == train_count_low) or (Xtrain_fin == train_count_high)) and ((ytrain_fin == train_count_low) or (ytrain_fin == train_count_high))):
        raise ValueError(f'Training set size should be approx. {train_size * X_len}, instead is: {X_train.shape[0]}')
    if not (((Xval_fin == val_count_low) or (Xval_fin == val_count_high)) and ((yval_fin == val_count_low) or (yval_fin == val_count_high))):
        raise ValueError(f'Validation set size should be approx. {val_size * X_len}, instead is: {X_val.shape[0]}')
    if not (((Xtest_fin == test_count_low) or (Xtest_fin == test_count_high)) and ((ytest_fin == test_count_low) or (ytest_fin == test_count_high))):
        raise ValueError(f'Test set size should be approx. {test_size * X_len}, instead is: {X_test.shape[0]}')

    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
def MLpipe_Stratified_Continous_r2(X, y, preprocessor, ML_algo, param_grid, xgb=False):
    '''
    This function splits the data to train, validation, and test (60/20/20).
    The R^2 is maximized in cross-validation.
    
    This function:
    1. Loops through 10 different random states
    2. Splits the data 60/20/20.
    3. Fits a model with the predefined Preprocessor, trains the model with each hyperparameter combination in param_grid
    4. Calculates the model's error on the test set on the model with the best hyperparameter combinations in param_grid
    5. Returns a list of 10 test scores and 10 best models
    '''
    
    # lists to be returned
    test_scores = []
    best_models = []
    test_Xs = []
    test_ys = []

    nr_states = 10
    for i in range(nr_states):
        rs = 28 * i
        print('Random State:', rs)

        # split
        X_train, y_train, X_val, y_val, X_test, y_test = stratified_continuous_split(X, y, train_size=0.6, val_size=0.2, test_size=0.2, random_state=rs)

        # preprocess
        X_train_prep = preprocessor.fit_transform(X_train)
        X_val_prep = preprocessor.transform(X_val)
        X_test_prep = preprocessor.transform(X_test)

        # final preprocess with Standard Scaler so that I can use the coefficients of linear models as global importance metrics
        final_scaler = StandardScaler()
        X_train_prep = final_scaler.fit_transform(X_train_prep)
        X_val_prep = final_scaler.transform(X_val_prep)
        X_test_prep = final_scaler.transform(X_test_prep)

        test_Xs.append(X_test_prep)
        test_ys.append(y_test)

        # train and perform cross-validation        
        models = []
        val_scores = []
        for p in range(len(ParameterGrid(param_grid))):
            params = ParameterGrid(param_grid)[p]
            # print(' ',params) # TEMPORARY

            if (xgb):
                clf = ML_algo.fit(X_train_prep, y_train, early_stopping_rounds=50, eval_set=[(X_val_prep, y_val)], verbose=False)
            else:
                clf = ML_algo.fit(X_train_prep, y_train)
            models.append(clf)
            y_val_pred = clf.predict(X_val_prep)
            val_scores.append(r2_score(y_val, y_val_pred))
            # print(' Validation R^2:', val_scores[-1]) # TEMPORARY

        # save results
        print('    Best Model Parameters:', ParameterGrid(param_grid)[np.argmax(val_scores)])
        print('    Validation Set R^2:', np.max(val_scores))
        best_model_this_rs = models[np.argmax(val_scores)]
        best_models.append(best_model_this_rs)
        y_test_pred = best_model_this_rs.predict(X_test_prep)
        test_score = r2_score(y_test, y_test_pred)
        test_scores.append(test_score)
        print('    Baseline R^2 (test set):', r2_score(y_test, pd.Series([2]*len(y_test))))
        print('    Test Set R^2:', test_score)
        
    return test_scores, best_models, test_Xs, test_ys

### Run and Cross Validate Several Models - R^2

In [ ]:
# keeping track of scores
models_r2s = pd.DataFrame(columns=['R^2', 'l1', 'l2', 'elastic net', 'random forest', 'SVR', 'XGBoost'])
models_r2s['R^2'] = pd.Series(['mean', 'std dev'])
models_r2s = models_r2s.set_index('R^2')

#### (1) Linear Regression with l1 Regularization

In [ ]:
lin_reg_l1 = Lasso(random_state=random_state)
l1_params = {
    'lasso__alpha': np.linspace(math.exp(-2), math.exp(2), 21)
}
l1_test_scores_r2, l1_best_models_r2, l1_test_Xs_r2, l1_test_ys_r2 = MLpipe_Stratified_Continous_r2(X, y, preprocessor=preprocessor, ML_algo=lin_reg_l1, param_grid=l1_params)

In [ ]:
print('****Metrics with l1 Linear Regression:****')
mean = np.mean(l1_test_scores_r2)
std = np.std(l1_test_scores_r2)
models_r2s['l1']['mean'] = mean
models_r2s['l1']['std dev'] = std
print('Mean R^2:', mean)
print('Std. Deviation of R^2:', std)

In [ ]:
file = open('../results/l1_r2.save', 'wb')
best_model_index = np.argmax(l1_test_scores_r2)
pickle.dump((l1_best_models_r2[best_model_index], l1_test_Xs_r2[best_model_index], l1_test_ys_r2[best_model_index]), file)
file.close()

#### (2) Linear Regression with l2 Regularization

In [ ]:
lin_reg_l2 = Ridge(random_state=random_state)
l2_params = {
    'ridge__alpha': np.linspace(math.exp(-2), math.exp(2), 21)
}
l2_test_scores_r2, l2_best_models_r2, l2_test_Xs_r2, l2_test_ys_r2 = MLpipe_Stratified_Continous_r2(X, y, preprocessor=preprocessor, ML_algo=lin_reg_l2, param_grid=l2_params)

In [ ]:
print('****Metrics with l2 Linear Regression:****')
mean = np.mean(l2_test_scores_r2)
std = np.std(l2_test_scores_r2)
models_r2s['l2']['mean'] = mean
models_r2s['l2']['std dev'] = std
print('Mean R^2:', mean)
print('Std. Deviation of R^2:', std)

In [ ]:
file = open('../results/l2_r2.save', 'wb')
best_model_index = np.argmax(l2_test_scores_r2)
pickle.dump((l2_best_models_r2[best_model_index], l2_test_Xs_r2[best_model_index], l2_test_ys_r2[best_model_index]), file)
file.close()

#### (3) Linear Regression with Elastic Net Regularization

In [ ]:
lin_reg_elastic = ElasticNet(random_state=random_state)
elastic_params = {
    'elasticnet__alpha': np.linspace(math.exp(-2), math.exp(2), 21),
    'elasticnet__l1_ratio': np.linspace(0, 1, 21)
}
elastic_test_scores_r2, elastic_best_models_r2, elastic_test_Xs_r2, elastic_test_ys_r2 = MLpipe_Stratified_Continous_r2(X, y, preprocessor=preprocessor, ML_algo=lin_reg_elastic, param_grid=elastic_params)

In [ ]:
print('****Metrics with Elastic Net Linear Regression:****')
mean = np.mean(elastic_test_scores_r2)
std = np.std(elastic_test_scores_r2)
models_r2s['elastic net']['mean'] = mean
models_r2s['elastic net']['std dev'] = std
print('Mean R^2:', mean)
print('Std. Deviation of R^2:', std)

In [ ]:
file = open('../results/elastic_net_r2.save', 'wb')
best_model_index = np.argmax(elastic_test_scores_r2)
pickle.dump((elastic_best_models_r2[best_model_index], elastic_test_Xs_r2[best_model_index], elastic_test_ys_r2[best_model_index]), file)
file.close()

#### (4) Random Forest Regressor

In [ ]:
random_forest_reg = RandomForestRegressor(n_jobs=-1, n_estimators=100, random_state=random_state)
rf_params = {
    'randomforestregressor__max_features': [1, 3, 10, 30],
    'randomforestregressor__max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}
rf_test_scores_r2, rf_best_models_r2, rf_test_Xs_r2, rf_test_ys_r2 = MLpipe_Stratified_Continous_r2(X, y, preprocessor=preprocessor, ML_algo=random_forest_reg, param_grid=rf_params)

In [ ]:
print('****Metrics with Random Forest Regressor:****')
mean = np.mean(rf_test_scores_r2)
std = np.std(rf_test_scores_r2)
models_r2s['random forest']['mean'] = mean
models_r2s['random forest']['std dev'] = std
print('Mean R^2:', mean)
print('Std. Deviation of R^2:', std)

In [ ]:
file = open('../results/random_forest_regressor_r2.save', 'wb')
best_model_index = np.argmax(rf_test_scores_r2)
pickle.dump((rf_best_models_r2[best_model_index], rf_test_Xs_r2[best_model_index], rf_test_ys_r2[best_model_index]), file)
file.close()

#### (5) SVR

In [ ]:
svr = SVR()
svr_params = {
    'svr__gamma': [1e-3, 1e-1, 1e1, 1e3, 1e5],
    'svr__C': [1e-1, 1e0, 1e1]
}
svr_test_scores_r2, svr_best_models_r2, svr_test_Xs_r2, svr_test_ys_r2 = MLpipe_Stratified_Continous_r2(X, y, preprocessor=preprocessor, ML_algo=svr, param_grid=svr_params)

In [ ]:
print('****Metrics with SVR:****')
mean = np.mean(svr_test_scores_r2)
std = np.std(svr_test_scores_r2)
models_r2s['SVR']['mean'] = mean
models_r2s['SVR']['std dev'] = std
print('Mean R^2:', mean)
print('Std. Deviation of R^2:', std)

In [ ]:
file = open('../results/svr_r2.save', 'wb')
best_model_index = np.argmax(svr_test_scores_r2)
pickle.dump((svr_best_models_r2[best_model_index], svr_test_Xs_r2[best_model_index], svr_test_ys_r2[best_model_index]), file)
file.close()

#### (6) XGBoost

In [ ]:
xgb = xgboost.XGBRegressor(seed=0, n_estimators=10000, learning_rate=0.03, colsample_bytree=0.9, subsample=0.66)
xgb_params = {
    'xgbregressor__reg_alpha': [0e0, 1e-2, 1e-1, 1e0, 1e1, 1e2],
    'xgbregressor__lambda': [0e0, 1e-2, 1e-1, 1e0, 1e1, 1e2],
    'xgbregressor__max_depth': [1, 3, 10, 30, 100]
}
xgb_test_scores_r2, xgb_best_models_r2, xgb_test_Xs_r2, xgb_test_ys_r2 = MLpipe_Stratified_Continous_r2(X, y, preprocessor=preprocessor, ML_algo=xgb, param_grid=xgb_params, xgb=True)

In [ ]:
print('****Metrics with XGBoost:****')
mean = np.mean(xgb_test_scores_r2)
std = np.std(xgb_test_scores_r2)
models_r2s['XGBoost']['mean'] = mean
models_r2s['XGBoost']['std dev'] = std
print('Mean R^2:', mean)
print('Std. Deviation of R^2:', std)

In [ ]:
file = open('../results/xgboost_r2.save', 'wb')
best_model_index = np.argmax(xgb_test_scores_r2)
pickle.dump((xgb_best_models_r2[best_model_index], xgb_test_Xs_r2[best_model_index], xgb_test_ys_r2[best_model_index]), file)
file.close()

### Results

In [ ]:
models_r2s